In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [2]:
img_labels = []
directory_path = Path('YogaPoses')
for file_path in directory_path.glob('*'):
    img_labels.append(str(file_path).split('\\')[-1])
    
labels_to_digits = {}
digits_to_labels = {}
for i, elem in enumerate(img_labels):
    labels_to_digits[elem] = i
    digits_to_labels[i] = elem
labels_to_digits, digits_to_labels

({'Downdog': 0, 'Goddess': 1, 'Plank': 2, 'Tree': 3, 'Warrior2': 4},
 {0: 'Downdog', 1: 'Goddess', 2: 'Plank', 3: 'Tree', 4: 'Warrior2'})

In [3]:
img_path = 'YogaPoses/'

# Словарь с названиями ключевых точек тела в MediaPipe Pose
# Заметьте, что точная структура может меняться в зависимости от версии MediaPipe, поэтому проверьте актуальность данных
pose_landmarks = {
    0: 'Нос',
    1: 'Внутренний глаз слева',
    2: 'Внутренний глаз справа',
    3: 'Внешний глаз слева',
    4: 'Внешний глаз справа',
    5: 'Левое ухо',
    6: 'Правое ухо',
    7: 'Рот слева',
    8: 'Рот справа',
    9: 'Левое плечо',
    10: 'Правое плечо',
    11: 'Левый локоть',
    12: 'Правый локоть',
    13: 'Левое запястье',
    14: 'Правое запястье',
    15: 'Левая ладонь',
    16: 'Правая ладонь',
    17: 'Левый мизинец',
    18: 'Правый мизинец',
    19: 'Левый указательный палец',
    20: 'Правый указательный палец',
    21: 'Левый большой палец',
    22: 'Правый большой палец',
    23: 'Левый бедро',
    24: 'Правое бедро',
    25: 'Левое колено',
    26: 'Правое колено',
    27: 'Левая щиколотка',
    28: 'Правая щиколотка',
    29: 'Левая пятка',
    30: 'Правая пятка',
    31: 'Левый передний край стопы',
    32: 'Правый передний край стопы',
}

In [4]:
directory_path = img_path

for elem in img_labels:
    if not os.path.isdir(directory_path+elem):
        print(f'Directory {directory_path+elem} is not exist')

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def work_with_file(file_path, df, label):
    img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)
    if results.pose_landmarks:
        landmarks = {'image_path': str(file_path)}
        
        for i, landmark in enumerate(results.pose_landmarks.landmark):
            landmarks[f'x_{i}'] = landmark.x
            landmarks[f'y_{i}'] = landmark.y
            landmarks[f'z_{i}'] = landmark.z
            landmarks[f'visibility_{i}'] = landmark.visibility
        landmarks['label'] = labels_to_digits[label]
        
        df = df._append(landmarks, ignore_index=True)
    
    return df

In [ ]:
df = pd.DataFrame()
for elem in img_labels:      
    n_files = 0
    directory_path = Path(img_path + elem)

    for file_path in directory_path.glob('*'):
        if file_path.is_file():
            n_files += 1
            df = work_with_file(str(file_path), df, elem)
            
    print(f'got {n_files} files from {directory_path} folder')
    
    
df.to_csv('body_landmarks.csv', index=False)
df